In [1]:
import pandas as pd
import os
import time
import sys
from datetime import datetime
from time import mktime
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")
import re
from Quandl import Quandl

auth_tok = "-EWms6UQtCVvQ8-drkxv"

%matplotlib inline


class Unbuffered(object):
   def __init__(self, stream):
       self.stream = stream
   def write(self, data):
       self.stream.write(data)
       self.stream.flush()
   def __getattr__(self, attr):
       return getattr(self.stream, attr)

sys.stdout = Unbuffered(sys.stdout)


In [3]:
path = "/Users/Shawn/GitHub/52weeks-bigfiles/8-stockprediction/intraQuarter" 

In [56]:
##STRUCTURE HISTORICAL DATA##
def Key_Stats (gather=["Total Debt/Equity",
                      'Trailing P/E',
                      'Price/Sales',
                      'Price/Book',
                      'Profit Margin',
                      'Operating Margin',
                      'Return on Assets',
                      'Return on Equity',
                      'Revenue Per Share',
                      'Market Cap',
                        'Enterprise Value',
                        'Forward P/E',
                        'PEG Ratio',
                        'Enterprise Value/Revenue',
                        'Enterprise Value/EBITDA',
                        'Revenue',
                        'Gross Profit',
                        'EBITDA',
                        'Net Income Avl to Common ',
                        'Diluted EPS',
                        'Earnings Growth',
                        'Revenue Growth',
                        'Total Cash',
                        'Total Cash Per Share',
                        'Total Debt',
                        'Current Ratio',
                        'Book Value Per Share',
                        'Cash Flow',
                        'Beta',
                        'Held by Insiders',
                        'Held by Institutions',
                        'Shares Short (as of',
                        'Short Ratio',
                        'Short % of Float',
                        'Shares Short (prior ']):
    
    statspath = path + '/_KeyStats'
    stock_list = [x[0] for x in os.walk(statspath)]
    df = pd.DataFrame(columns = ['Date',
                                 'Unix',
                                 'Ticker',
                                 'Price',
                                 'stock_p_change',
                                 'SP500',
                                 'sp500_p_change',
                                 'Difference',
                                 ##############
                                 'DE Ratio',
                                 'Trailing P/E',
                                 'Price/Sales',
                                 'Price/Book',
                                 'Profit Margin',
                                 'Operating Margin',
                                 'Return on Assets',
                                 'Return on Equity',
                                 'Revenue Per Share',
                                 'Market Cap',
                                 'Enterprise Value',
                                 'Forward P/E',
                                 'PEG Ratio',
                                 'Enterprise Value/Revenue',
                                 'Enterprise Value/EBITDA',
                                 'Revenue',
                                 'Gross Profit',
                                 'EBITDA',
                                 'Net Income Avl to Common ',
                                 'Diluted EPS',
                                 'Earnings Growth',
                                 'Revenue Growth',
                                 'Total Cash',
                                 'Total Cash Per Share',
                                 'Total Debt',
                                 'Current Ratio',
                                 'Book Value Per Share',
                                 'Cash Flow',
                                 'Beta',
                                 'Held by Insiders',
                                 'Held by Institutions',
                                 'Shares Short (as of',
                                 'Short Ratio',
                                 'Short % of Float',
                                 'Shares Short (prior ',                                
                                 ##############
                                 'Status'])
    
    sp500_df = pd.DataFrame.from_csv("YAHOO-INDEX_GSPC.csv")
    stock_df = pd.DataFrame.from_csv("stock_prices.csv")
    
    ticker_list = []
    

    for each_dir in stock_list[1:]:
        each_file = os.listdir(each_dir)
        ticker = each_dir.split('/Users/Shawn/GitHub/52weeks-bigfiles/8-stockprediction/intraQuarter/_KeyStats/')[1]
        ticker_list.append(ticker)
        
#         starting_stock_value = False
#         starting_sp500_value = False
        
        if len(each_file) > 0:
            for file in each_file:
                date_stamp = datetime.strptime(file, '%Y%m%d%H%M%S.html')
                unix_time = time.mktime(date_stamp.timetuple())
                full_file_path = each_dir + '/' + file
                source = open(full_file_path,'r').read()
                try:
                    value_list = []
                    
                    for each_data in gather:    
                        try:
                            regex = re.escape(each_data) + r'.*?(\d{1,8}\.\d{1,8}M?B?|N/A)%?</td>'
                            value = re.search(regex , source)
                            value = (value.group(1))
                            
                            if "B" in value:
                                value = float(value.replace("B",''))*1000000000
                            elif "M" in value: 
                                value = float(value.replace("M",''))*1000000
                            
                            value_list.append(value)

                            
                        except Exception as e:
                            value = "N/A"
                            value_list.append(value)
                    try:
                        sp500_date = datetime.fromtimestamp(unix_time).strftime('%Y-%m-%d')
                        row = sp500_df[(sp500_df.index == sp500_date)]
                        sp500_value = float(row["Adj Close"])
                    except:
                        try:
                            sp500_date = datetime.fromtimestamp(unix_time-259200).strftime('%Y-%m-%d')
                            row = sp500_df[(sp500_df.index == sp500_date)]
                            sp500_value = float(row["Adj Close"])
                        except Exception as e:
                            print "fapsdolkfhasf;lsak",str(e)


                    one_year_later = int(unix_time + 31536000)

                    try:
                        sp500_1y = datetime.fromtimestamp(one_year_later).strftime('%Y-%m-%d')
                        row = sp500_df[(sp500_df.index == sp500_1y)]
                        sp500_1y_value = float(row["Adj Close"])
                    except:
                        try:
                            sp500_1y = datetime.fromtimestamp(one_year_later-259200).strftime('%Y-%m-%d')
                            row = sp500_df[(sp500_df.index == sp500_1y)]
                            sp500_1y_value = float(row["Adj Close"])
                        except Exception as e:
                            print "sp500 1 year later issue",str(e)



                    try:
                        stock_price_1y = datetime.fromtimestamp(one_year_later).strftime('%Y-%m-%d')
                        row = stock_df[(stock_df.index == stock_price_1y)][ticker.upper()]
                        

                        stock_1y_value = round(float(row),2)
##                        print(stock_1y_value)
##                        time.sleep(1555)
                    
                    except Exception as e:
                        try:
                            stock_price_1y = datetime.fromtimestamp(one_year_later-259200).strftime('%Y-%m-%d')
                            row = stock_df[(stock_df.index == stock_price_1y)][ticker.upper()]
                            stock_1y_value = round(float(row),2)
                        except Exception as e:
                            print "stock price:",str(e)




                    try:
                        stock_price = datetime.fromtimestamp(unix_time).strftime('%Y-%m-%d')
                        row = stock_df[(stock_df.index == stock_price)][ticker.upper()]
                        stock_price = round(float(row),2)
                    
                    except Exception as e:
                        try:
                            stock_price = datetime.fromtimestamp(unix_time-259200).strftime('%Y-%m-%d')
                            row = stock_df[(stock_df.index == stock_price)][ticker.upper()]
                            stock_price = round(float(row),2)
                        except Exception as e:
                            print "stock price:",str(e)
    
                        
                    stock_p_change = round(((stock_1y_value - stock_price ) / stock_price * 100),2)
                    sp500_p_change = round(((sp500_1y_value - sp500_value ) / sp500_value * 100),2)
                    
                    difference = stock_p_change - sp500_p_change
                    if difference > 5:
                        status = 1
                    else:
                        status = 0
                        
                    if value_list.count("N/A") > 15:
                        pass
                    else:
                        df = df.append({'Date':date_stamp,
                                            'Unix':unix_time,
                                            'Ticker':ticker,
                                            
                                            'Price':stock_price,
                                            'stock_p_change':stock_p_change,
                                            'SP500':sp500_value,
                                            'sp500_p_change':sp500_p_change,
                                            'Difference':difference,
                                            'DE Ratio':value_list[0],
                                            #'Market Cap':value_list[1],
                                            'Trailing P/E':value_list[1],
                                            'Price/Sales':value_list[2],
                                            'Price/Book':value_list[3],
                                            'Profit Margin':value_list[4],
                                            'Operating Margin':value_list[5],
                                            'Return on Assets':value_list[6],
                                            'Return on Equity':value_list[7],
                                            'Revenue Per Share':value_list[8],
                                            'Market Cap':value_list[9],
                                             'Enterprise Value':value_list[10],
                                             'Forward P/E':value_list[11],
                                             'PEG Ratio':value_list[12],
                                             'Enterprise Value/Revenue':value_list[13],
                                             'Enterprise Value/EBITDA':value_list[14],
                                             'Revenue':value_list[15],
                                             'Gross Profit':value_list[16],
                                             'EBITDA':value_list[17],
                                             'Net Income Avl to Common ':value_list[18],
                                             'Diluted EPS':value_list[19],
                                             'Earnings Growth':value_list[20],
                                             'Revenue Growth':value_list[21],
                                             'Total Cash':value_list[22],
                                             'Total Cash Per Share':value_list[23],
                                             'Total Debt':value_list[24],
                                             'Current Ratio':value_list[25],
                                             'Book Value Per Share':value_list[26],
                                             'Cash Flow':value_list[27],
                                             'Beta':value_list[28],
                                             'Held by Insiders':value_list[29],
                                             'Held by Institutions':value_list[30],
                                             'Shares Short (as of':value_list[31],
                                             'Short Ratio':value_list[32],
                                             'Short % of Float':value_list[33],
                                             'Shares Short (prior ':value_list[34],
                                            'Status':status},
                                           ignore_index=True)
                except Exception as e:
                    pass
                
#     for each_ticker in ticker_list:
#         try:
#             plot_df = df[(df['Ticker'] == each_ticker)]
#             plot_df = plot_df.set_index(['Date'])
            
#             if plot_df['Status'][-1] == "underperform":
#                 color = 'r'
#             else:
#                 color = 'g'
            
#             plot_df['Difference'].plot(label = each_ticker, color = color)
#             plt.legend()
#             plt.axes()
            
#         except:
#             pass
#     plt.show()

    df.to_csv("key_stats_acc_perf_WITH_NA_enhanced.csv")
    
        
Key_Stats()


sp500 1 year later issue cannot convert the series to <type 'float'>
sp500 1 year later issue cannot convert the series to <type 'float'>
fapsdolkfhasf;lsak cannot convert the series to <type 'float'>
stock price: cannot convert the series to <type 'float'>
fapsdolkfhasf;lsak cannot convert the series to <type 'float'>
fapsdolkfhasf;lsak cannot convert the series to <type 'float'>
stock price: cannot convert the series to <type 'float'>
stock price: 'ACT'
stock price: 'ACT'
fapsdolkfhasf;lsak cannot convert the series to <type 'float'>
stock price: cannot convert the series to <type 'float'>
fapsdolkfhasf;lsak cannot convert the series to <type 'float'>
stock price: cannot convert the series to <type 'float'>
sp500 1 year later issue cannot convert the series to <type 'float'>
sp500 1 year later issue cannot convert the series to <type 'float'>
stock price: cannot convert the series to <type 'float'>
fapsdolkfhasf;lsak cannot convert the series to <type 'float'>
stock price: cannot con

In [48]:
##STRUCTURE FORWARD DATA##
def Forward(gather=["Total Debt/Equity",
                      'Trailing P/E',
                      'Price/Sales',
                      'Price/Book',
                      'Profit Margin',
                      'Operating Margin',
                      'Return on Assets',
                      'Return on Equity',
                      'Revenue Per Share',
                      'Market Cap',
                        'Enterprise Value',
                        'Forward P/E',
                        'PEG Ratio',
                        'Enterprise Value/Revenue',
                        'Enterprise Value/EBITDA',
                        'Revenue',
                        'Gross Profit',
                        'EBITDA',
                        'Net Income Avl to Common ',
                        'Diluted EPS',
                        'Earnings Growth',
                        'Revenue Growth',
                        'Total Cash',
                        'Total Cash Per Share',
                        'Total Debt',
                        'Current Ratio',
                        'Book Value Per Share',
                        'Cash Flow',
                        'Beta',
                        'Held by Insiders',
                        'Held by Institutions',
                        'Shares Short (as of',
                        'Short Ratio',
                        'Short % of Float',
                        'Shares Short (prior ']):
    
##    statspath = path+'/_KeyStats'
##    stock_list = [x[0] for x in os.walk(statspath)]
    df = pd.DataFrame(columns = ['Date',
                                 'Unix',
                                 'Ticker',
                                 'Price',
                                 'stock_p_change',
                                 'SP500',
                                 'sp500_p_change',
                                 'Difference',
                                 ##############
                                 'DE Ratio',
                                 'Trailing P/E',
                                 'Price/Sales',
                                 'Price/Book',
                                 'Profit Margin',
                                 'Operating Margin',
                                 'Return on Assets',
                                 'Return on Equity',
                                 'Revenue Per Share',
                                 'Market Cap',
                                 'Enterprise Value',
                                 'Forward P/E',
                                 'PEG Ratio',
                                 'Enterprise Value/Revenue',
                                 'Enterprise Value/EBITDA',
                                 'Revenue',
                                 'Gross Profit',
                                 'EBITDA',
                                 'Net Income Avl to Common ',
                                 'Diluted EPS',
                                 'Earnings Growth',
                                 'Revenue Growth',
                                 'Total Cash',
                                 'Total Cash Per Share',
                                 'Total Debt',
                                 'Current Ratio',
                                 'Book Value Per Share',
                                 'Cash Flow',
                                 'Beta',
                                 'Held by Insiders',
                                 'Held by Institutions',
                                 'Shares Short (as of',
                                 'Short Ratio',
                                 'Short % of Float',
                                 'Shares Short (prior ',                                
                                 ##############
                                 'Status'])


    file_list = os.listdir("forward")

    for each_file in file_list:

        ticker = each_file.split(".html")[0]
        full_file_path = "forward/"+each_file
        source = open(full_file_path,"r").read()


        

        try:
            value_list = []

            for each_data in gather:
                try:
                    regex = re.escape(each_data) + r'.*?(\d{1,8}\.\d{1,8}M?B?|N/A)%?'
                    value = re.search(regex, source)
                    value = (value.group(1))

                    if "B" in value:
                        value = float(value.replace("B",''))*1000000000

                    elif "M" in value:
                        value = float(value.replace("M",''))*1000000

                    value_list.append(value)
                    
                    
                except Exception as e:
                    value = "N/A"
                    value_list.append(value)

            if value_list.count("N/A") > 0:
                pass
            else:
                

                df = df.append({'Date':"N/A",
                                    'Unix':"N/A",
                                    'Ticker':ticker,
                                    
                                    'Price':"N/A",
                                    'stock_p_change':"N/A",
                                    'SP500':"N/A",
                                    'sp500_p_change':"N/A",
                                    'Difference':"N/A",
                                    'DE Ratio':value_list[0],
                                    #'Market Cap':value_list[1],
                                    'Trailing P/E':value_list[1],
                                    'Price/Sales':value_list[2],
                                    'Price/Book':value_list[3],
                                    'Profit Margin':value_list[4],
                                    'Operating Margin':value_list[5],
                                    'Return on Assets':value_list[6],
                                    'Return on Equity':value_list[7],
                                    'Revenue Per Share':value_list[8],
                                    'Market Cap':value_list[9],
                                     'Enterprise Value':value_list[10],
                                     'Forward P/E':value_list[11],
                                     'PEG Ratio':value_list[12],
                                     'Enterprise Value/Revenue':value_list[13],
                                     'Enterprise Value/EBITDA':value_list[14],
                                     'Revenue':value_list[15],
                                     'Gross Profit':value_list[16],
                                     'EBITDA':value_list[17],
                                     'Net Income Avl to Common ':value_list[18],
                                     'Diluted EPS':value_list[19],
                                     'Earnings Growth':value_list[20],
                                     'Revenue Growth':value_list[21],
                                     'Total Cash':value_list[22],
                                     'Total Cash Per Share':value_list[23],
                                     'Total Debt':value_list[24],
                                     'Current Ratio':value_list[25],
                                     'Book Value Per Share':value_list[26],
                                     'Cash Flow':value_list[27],
                                     'Beta':value_list[28],
                                     'Held by Insiders':value_list[29],
                                     'Held by Institutions':value_list[30],
                                     'Shares Short (as of':value_list[31],
                                     'Short Ratio':value_list[32],
                                     'Short % of Float':value_list[33],
                                     'Shares Short (prior ':value_list[34],
                                    'Status':"N/A"},
                                   ignore_index=True)
        except Exception as e:
            pass



    df.to_csv("forward_sample_NO_NA.csv")
    
        

    


Forward()

In [71]:
##LEARNING WITH FORWARD##
import numpy as np
from sklearn import svm, preprocessing
from __future__ import division
import warnings
from collections import Counter

warnings.filterwarnings("ignore", category=DeprecationWarning) 

how_much_better = 2

FEATURES =  ['DE Ratio',
             'Trailing P/E',
             'Price/Sales',
             'Price/Book',
             'Profit Margin',
             'Operating Margin',
             'Return on Assets',
             'Return on Equity',
             'Revenue Per Share',
             'Market Cap',
             'Enterprise Value',
             'Forward P/E',
             'PEG Ratio',
             'Enterprise Value/Revenue',
             'Enterprise Value/EBITDA',
             'Revenue',
             'Gross Profit',
             'EBITDA',
             'Net Income Avl to Common ',
             'Diluted EPS',
             'Earnings Growth',
             'Revenue Growth',
             'Total Cash',
             'Total Cash Per Share',
             'Total Debt',
             'Current Ratio',
             'Book Value Per Share',
             'Cash Flow',
             'Beta',
             'Held by Insiders',
             'Held by Institutions',
             'Shares Short (as of',
             'Short Ratio',
             'Short % of Float',
             'Shares Short (prior ']

def Status_Calc(stock, sp500):
    difference = stock - sp500
    if difference > how_much_better:
        return 1
    else:
        return 0

def Build_Data_Set():
    data_df = pd.DataFrame.from_csv("key_stats_acc_perf_NO_NA_enhanced.csv")

    #data_df = data_df[:100]
    data_df = data_df.reindex(np.random.permutation(data_df.index))
    data_df = data_df.replace("NaN",0).replace("N/A",0)
    
    data_df["Status2"] = list(map(Status_Calc,data_df["stock_p_change"],data_df["sp500_p_change"]))

    X = np.array(data_df[FEATURES].values)
    

    y = (data_df["Status2"]
         .replace("underperform",0)
         .replace("outperform",1)
         .values.tolist())
    
    
    X = preprocessing.scale(X)
    
    Z = np.array(data_df[["stock_p_change","sp500_p_change"]])


    return X,y,Z

def Analysis():
    
    test_size = 100
    
    invest_amount = 10000
    total_invests = 0
    if_market = 0
    if_strat = 0
    
    X, y, Z = Build_Data_Set()
    print len(X)
    
    
    clf = svm.SVC(kernel="linear", C= 1.0)
    clf.fit(X[:-test_size],y[:-test_size])
              
    correct_count = 0
    
    for x in range(1, test_size + 1):
        if clf.predict(X[-x])[0] == y[-x]:
            correct_count += 1
            
        if clf.predict(X[-x])[0] == 1:
            invest_return = invest_amount + (invest_amount * (Z[-x][0]/100))
            market_return = invest_amount + (invest_amount * (Z[-x][1]/100))
            
            total_invests += 1
            if_market += market_return
            if_strat += invest_return
            
            
    print "Accuracy: " , correct_count / test_size *100.00
    print "Total Trades: " , total_invests
    print "Ending with Strategy: ", if_strat
    print "Ending with Market: " , if_market
    
    compared = (if_strat - if_market)/if_market *100
    
    do_nothing = total_invests * invest_amount
    
    avg_market = (if_market - do_nothing) / do_nothing * 100
    avg_strat = (if_strat - do_nothing) / do_nothing * 100
    
    print "Compared to market we earn" , str(compared)+"% more"
    print "Average strategy return: ", str(avg_strat)+"%"
    print "Average market return: ", str(avg_market)+"%"
    
    data_df = pd.DataFrame.from_csv("forward_sample_NO_NA.csv")
    data_df = data_df.replace("N/A",0).replace("NaN",0)
    
    X = np.array(data_df[FEATURES].values)    
    
    X = preprocessing.scale(X)
    
    Z = data_df["Ticker"].values.tolist()
    
    invest_list = []
    
    for i in range(len(X)):
        p = clf.predict(X[i])[0]
        if p == 1:
#             print Z[i]
            invest_list.append(Z[i])
#     print len(invest_list)
#     print invest_list
    return invest_list

final_list = []

loops = 5

for x in range(loops):
    stock_list = Analysis()
    for e in stock_list:
        final_list.append(e)
        
x = Counter(final_list)

print 15*"_"
for each in x:
    if x[each] > loops - (loops/3):
        print(each)

    
Analysis()

2965
Accuracy:  60.0
Total Trades:  62
Ending with Strategy:  813877.0
Ending with Market:  696891.0
Compared to market we earn 16.7868432797% more
Average strategy return:  31.270483871%
Average market return:  12.4017741935%
2965
Accuracy:  64.0
Total Trades:  58
Ending with Strategy:  707957.0
Ending with Market:  651190.0
Compared to market we earn 8.71742502188% more
Average strategy return:  22.0615517241%
Average market return:  12.274137931%
2965
Accuracy:  54.0
Total Trades:  60
Ending with Strategy:  698244.0
Ending with Market:  667632.0
Compared to market we earn 4.58516068732% more
Average strategy return:  16.374%
Average market return:  11.272%
2965
Accuracy:  54.0
Total Trades:  59
Ending with Strategy:  683242.0
Ending with Market:  667763.0
Compared to market we earn 2.31803798653% more
Average strategy return:  15.8037288136%
Average market return:  13.1801694915%
2965
Accuracy:  56.0
Total Trades:  62
Ending with Strategy:  721881.0
Ending with Market:  699812.0
Com

['a',
 'aci',
 'adi',
 'adp',
 'adt',
 'aet',
 'afl',
 'akam',
 'all',
 'amat',
 'amgn',
 'amp',
 'anf',
 'aon',
 'apd',
 'aph',
 'avy',
 'ba',
 'bax',
 'bbby',
 'bby',
 'ben',
 'biib',
 'blk',
 'bms',
 'brcm',
 'bwa',
 'ca',
 'cab',
 'cah',
 'cam',
 'camp',
 'cf',
 'ci',
 'cinf',
 'cme',
 'cmi',
 'cms',
 'coh',
 'cost',
 'csc',
 'csco',
 'ctas',
 'ctxs',
 'dal',
 'dds',
 'dg',
 'dgx',
 'dhi',
 'dhr',
 'disca',
 'dks',
 'dlph',
 'dlx',
 'do',
 'dov',
 'dps',
 'dva',
 'ebay',
 'ecl',
 'efx',
 'el',
 'emc',
 'esrx',
 'ew',
 'exc',
 'expe',
 'expr',
 'fdx',
 'fis',
 'fl',
 'flir',
 'flr',
 'fls',
 'fosl',
 'fslr',
 'fti',
 'gd',
 'ges',
 'glw',
 'gm',
 'gme',
 'gpc',
 'gt',
 'gww',
 'har',
 'has',
 'hcp',
 'hig',
 'hot',
 'hst',
 'hum',
 'ibm',
 'ice',
 'iff',
 'ipg',
 'ir',
 'irm',
 'ivz',
 'jbl',
 'jci',
 'jec',
 'jwn',
 'kmx',
 'kr',
 'kss',
 'ksu',
 'leg',
 'lly',
 'lm',
 'lnc',
 'lrcx',
 'luv',
 'm',
 'mat',
 'mchp',
 'mck',
 'mdlz',
 'met',
 'mhfi',
 'mmc',
 'mos',
 'mu',
 'ndaq',
 

9146
a
aa
abc
abt
aci
acn
adbe
adi
adp
adsk
adt
aee
aeo
aep
aes
aet
afl
agn
aig
aiv
aiz
akam
all
alxn
amat
amd
amgn
amp
amt
an
anf
aon
apa
apc
apd
aph
apol
arg
arna
aro
ati
atvi
avb
avp
avy
axp
azo
ba
bac
bax
bbby
bbry
bbt
bby
bcr
bdx
ben
bf-b
bhi
big
biib
bk
bks
blk
bll
bms
bmy
brcm
brk-b
bsx
btu
bwa
bxp
c
ca
cab
cag
cah
cam
camp
cat
cb
cbg
cbs
cce
cci
ccl
celg
cern
cf
chk
chrw
ci
cim
cinf
cl
cldx
cma
cmcsa
cme
cmg
cmi
cms
cnp
cnx
cof
cog
coh
col
cop
cost
cpb
crm
csc
csco
csx
ctas
ctl
ctsh
ctxs
cvc
cvs
cvx
d
dal
dd
dds
de
df
dfs
dg
dgx
dhi
dhr
dis
disca
dks
dlph
dltr
dlx
dnb
dnr
do
dov
dow
dps
dri
dsw
dte
duk
dva
dvn
ea
ebay
ecl
ecyt
ed
efx
eix
el
emc
emn
emr
eog
eqr
eqt
esrx
esv
etfc
etn
etr
ew
exc
expd
expe
expr
fast
fb
fcx
fdx
fe
ffiv
fhn
fis
fisv
fitb
fl
flir
flr
fls
flws
fmc
fosl
fslr
fti
ftr
gas
gci
gd
ges
gild
gis
glw
gm
gmcr
gme
gnw
gpc
gps
grmn
grpn
gt
gtn
gww
hal
har
has
hban
hcn
hcp
hd
hes
hig
hog
hon
hot
hov
hp
hpq
hrb
hrl
hrs
hst
hsy
htz
hum
ibm
ice
iff
igt
intc
intu
ip
i

In [8]:
##DOWNLOAD FROM QUANDL##

def Stock_Prices():
    df = pd.DataFrame()
    statspath = path + "/_KeyStats"
    stock_list = [x[0] for x in os.walk(statspath)]
    
    for each_dir in stock_list[1:]:
        try:
            ticker = each_dir.split('/Users/Shawn/GitHub/52weeks-bigfiles/8-stockprediction/intraQuarter/_KeyStats/')[1]
    
            name = "WIKI/"+ticker.upper()
            data = Quandl.get(name,
                             trim_start = "2000-12-12", 
                              trim_end = "2014-12-30", 
                              authtoken = auth_tok)
            data[ticker.upper()] = data["Adj. Close"]
            df = pd.concat([df,data[ticker.upper()]], axis = 1)
        except Exception as e:
            print str(e)
            
    df.to_csv("stock_prices.csv")
    
Stock_Prices()

Dataset not found. Check Quandl code: WIKI/ACT for errors
Dataset not found. Check Quandl code: WIKI/BBRY for errors
Error Downloading! HTTP Error 400: Bad Request
Dataset not found. Check Quandl code: WIKI/BMC for errors
Error Downloading! HTTP Error 400: Bad Request
Dataset not found. Check Quandl code: WIKI/JOSB for errors
Dataset not found. Check Quandl code: WIKI/LTD for errors
Dataset not found. Check Quandl code: WIKI/MOLX for errors
Dataset not found. Check Quandl code: WIKI/NOK for errors
Dataset not found. Check Quandl code: WIKI/NYX for errors
Dataset not found. Check Quandl code: WIKI/SAI for errors
Dataset not found. Check Quandl code: WIKI/SKS for errors
Dataset not found. Check Quandl code: WIKI/SNE for errors
Dataset not found. Check Quandl code: WIKI/SNTS for errors
Dataset not found. Check Quandl code: WIKI/TM for errors
Dataset not found. Check Quandl code: WIKI/VALE for errors
Dataset not found. Check Quandl code: WIKI/WPO for errors


In [42]:
##DOWNLOAD FROM YAHOO##
import urllib2
from urllib2 import Request

def Check_Yahoo():
    statspath = path+"/_KeyStats"
    stock_list = [x[0] for x in os.walk(statspath)]
    
    for e in stock_list[1:]:

        time.sleep(1)
        try:
            e = e.replace(path + '/_KeyStats/','')
            link = "http://finance.yahoo.com/q/ks?s="+e.upper()+"+Key+Statistics"
            req = Request(link)
            resp = urllib2.urlopen(req).read()
            
            
            save = "forward/"+str(e)+".html"
            store = open(save,"w")
            store.write(str(resp))
            store.close()
            
        except Exception as e:
            print str(e)
            time.sleep(1)

Check_Yahoo()

In [ ]:
##PART 11 Tutorial##
import numpy as np
from sklearn import svm



X = np.array([[1,2],
             [5,8],
             [1.5,1.8],
             [8,8],
             [1,.6],
             [9,11]])
y = [0,1,0,1,0,1]

clf = svm.SVC(kernel = 'linear', C = 1.0)
clf.fit(X,y)

print clf.predict([10.58,10.76])

w = clf.coef_[0]

print w

a = -w[0] / w[1]

xx= np.linspace(0,12)
yy = a * xx - clf.intercept_[0] / w[1]

h0 = plt.plot(xx, yy, 'k-', label="non weighted div")

plt.scatter(X[:,0], X[:,1], c = y)
plt.legend()
plt.show()

